In [2]:
import numpy as np
import scipy as sp 
from matplotlib import pyplot as plt

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
# define relevant constants
n = 1000
m = 50
sigma_2 = 5
sigma = np.sqrt(sigma_2)
m = 9
k = 10

In [5]:
# generate sample data 
for i in range(n):
    for j in range(m):
        noise = np.random.normal(0,sigma,n)
        x = np.random.uniform(-1,1,size=n)
        y = np.sin(2 * np.pi * x) + noise


array([-5.28037327e-01, -6.31198107e-01, -6.67292398e-01,  2.76003204e-01,
        1.01782353e-01, -1.42468174e-01,  2.89800916e-01, -5.67541630e-01,
        5.78726937e-01,  2.07601709e-01,  7.90838128e-01, -4.86379101e-01,
        4.50837699e-01, -1.68996331e-01, -7.93201578e-01,  4.12458154e-02,
        3.71083554e-02,  2.30446862e-02,  3.22463771e-01,  4.96189962e-01,
        8.36129132e-01,  1.59219674e-02, -5.96428404e-01, -1.33242786e-02,
       -9.54902044e-01,  9.68337121e-01, -5.27747284e-01,  7.09665666e-01,
       -4.54961220e-01,  4.53215495e-02,  3.21844279e-01, -3.92183554e-01,
       -1.67869646e-01, -6.54403896e-01,  7.36518477e-01,  9.34057994e-01,
       -9.43965279e-01, -4.47664670e-01, -6.02294117e-01,  8.98297083e-01,
       -9.67550578e-01,  3.82626823e-01, -9.85337544e-01, -5.36515122e-01,
       -9.48630510e-01,  2.78292161e-01, -8.29052302e-01, -5.98030914e-01,
        7.56379193e-01,  9.20772607e-01,  7.04607731e-01,  2.46846761e-01,
        1.43084993e-01, -

(1000,)

In [4]:
# create design matrix function, polyreg function, k-fold training function
x = np.vstack(
# recursive function for polynomial design matrix; yes, this is not memoized. should do this in future
def designmatrix(x_input,p):
    if (p == 1):
        X = np.vstack([np.ones(np.size(x_input)), x_input]).T
        return X
    else:
        X = np.vstack([designmatrix(x_input, p - 1).T, x_input**p]).T
        return X

# function to perform polynomial regression
def polyreg(x_input, y_input, p):
    X = designmatrix(x_input, p)
    w = np.linalg.lstsq(X,y_input,rcond=None)[0]
    return w, X

def kfold(x_input, y_input, p):
    
    return x_train, y_train, x_test, y_test


In [8]:
# call the functions!

for i in range(1, m+1):
    for j in range(1, k+1):
        x_train, y_train, x_test, y_test = kfold(x,y,j)
        w, X = polyreg(x_train, y_train, i)
        
        
        
    

TypeError: unhashable type: 'numpy.ndarray'